In [1]:
from __future__ import print_function
from functools import reduce
import re
from nltk.tokenize import word_tokenize
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import load_model
import json
from pprint import pprint
import numpy as np
import csv
import string

from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [4]:
with open('train.json') as data_file:    
    data = json.load(data_file)
ids = []
titles = dict()
contexts = list()
questions = list()
contexts_uni = dict()
questions_uni = dict()
answers_text = list()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
#             answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts.append(context)
            contexts_uni[id_] = context
#             answers_start[id_] = answer["answer_start"]
#             answers_text.append(answer["text"])  
            questions.append(question)
            questions_uni[id_] = question

max_para = 672
max_q = 50
dimension = 300


In [5]:
t = Tokenizer()
t.fit_on_texts(contexts+questions) 
vocab_size = len(t.word_index) + 1
print(vocab_size)

74303


In [6]:
with open('test.json') as data_file:    
    data = json.load(data_file)
ids = []
titles = dict()
contexts = list()
questions = list()
contexts_uni = dict()
questions_uni = dict()
answers_text = list()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
#             answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts.append(context)
            contexts_uni[id_] = context
#             answers_start[id_] = answer["answer_start"]
#             answers_text.append(answer["text"])  
            questions.append(question)
            questions_uni[id_] = question
train_len = len(questions)
l = [i for i in range(len(ids))]
# l.sort(key=lambda x: np.random.random())
ind = l[:train_len]
print(train_len)

36790


In [7]:
para_vocab_size = len(t.word_index) + 1
encoded_para = t.texts_to_sequences(contexts[0:train_len])
padded_para = pad_sequences(encoded_para, maxlen=max_para, padding='post')
embedding_para = zeros((para_vocab_size, dimension))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_para[i] = embedding_vector

In [8]:
qns_vocab_size = len(t.word_index) + 1
encoded_qns = t.texts_to_sequences(questions[0:train_len])
padded_qns = pad_sequences(encoded_qns, maxlen=max_q, padding='post')
embedding_qns = zeros((qns_vocab_size, dimension))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_qns[i] = embedding_vector

In [9]:
model = load_model('NLP_first.h5')

In [10]:
out = model.predict([padded_para,padded_qns])

In [11]:
out_1 = np.argmax(out[0], axis=1)
out_2 = np.argmax(out[1], axis=1)

In [12]:
def map_word(em_words):
    index_word = {v: k for k, v in t.word_index.items()} # map back
    words = []
    for word in em_words:
        if len(em_words):
            if word[0] != 0:
                words.append(index_word.get(word[0]))
        else:
            words.append(' ')
    return (' '.join(words)) # output

In [13]:
# i=10000
# print(out_1[i])
# print(out_2[i])
# seqs = t.texts_to_sequences(contexts[0])
# print(np.shape(seqs[0][0]))
# a = padded_para[i][out_1[i]]
# print (type(seqs))
# b = a.reshape(1,1).tolist()
# print(type(b))
# ans = map_word(b)
# print(ans)
ans = list()
for i in range(train_len):
    s = out_1[i]
    e = out_2[i]
    if s < e:
        a = padded_para[i][s:e]
        cur_ans = a.reshape(len(a), 1).tolist()
        ans.append(map_word(cur_ans))
    elif s > e:
        a = padded_para[i][e:s]
        cur_ans = a.reshape(len(a), 1).tolist()
        ans.append(map_word(cur_ans))
    else: #s == e
        a = padded_para[i][s]
        cur_ans = a.reshape(1, 1).tolist()
        ans.append(map_word(cur_ans))
print(len(ans))

36790


In [14]:
for i in range(100):
    print(contexts[i])
    print()
    print(questions[i])
    print()
    print(normalize_answer(ans[i]))
    print('-----------------------')
    

Burke was born in Dublin, Ireland. His mother Mary née Nagle (c. 1702 – 1770) was a Roman Catholic who hailed from a déclassé County Cork family (and a cousin of Nano Nagle), whereas his father, a successful solicitor, Richard (died 1761), was a member of the Church of Ireland; it remains unclear whether this is the same Richard Burke who converted from Catholicism. The Burke dynasty descends from an Anglo-Norman knight surnamed de Burgh (latinised as de Burgo) who arrived in Ireland in 1185 following Henry II of England's 1171 invasion of Ireland.

Where was Burke born?

dublin
-----------------------
Burke was born in Dublin, Ireland. His mother Mary née Nagle (c. 1702 – 1770) was a Roman Catholic who hailed from a déclassé County Cork family (and a cousin of Nano Nagle), whereas his father, a successful solicitor, Richard (died 1761), was a member of the Church of Ireland; it remains unclear whether this is the same Richard Burke who converted from Catholicism. The Burke dynasty des

In [15]:
with open("submission_new.csv", 'w', encoding = 'utf-8') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['Id','Answer'])
    for i in ind:
        writer.writerow([ids[i],normalize_answer(ans[i])])
     